In [6]:
from   dst_predict.imports import etl
from   dst_predict.imports import window

import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp
from   jaxkan.models.KAN import KAN
from   flax import nnx
from   sklearn.model_selection import train_test_split
from   sklearn.metrics import mean_squared_error
import optax
import pandas as pd

In [7]:
file = open("../datasets/WWW_dstae01508718.dat", 'r')
records = etl.read_records(file)
rec = None
with file as f:
    rec = next(records)

rec["data"].size

744

In [8]:
# Training from data in 64 datapoints:
# 64 / 24 = around 2.6 days of geo magnetic data
training_sets = window.training(rec, 0)

df_inputs = pd.DataFrame(training_sets["inputs"])

df_data = df_inputs[0][0]
df_data = pd.DataFrame.from_dict(df_data, orient="index").transpose()

print(df_inputs)
df_data

                                                   0
0  {'dst_nT': [-45.0, -37.0, -37.0, -41.0, -45.0,...
1  {'dst_nT': [-37.0, -37.0, -41.0, -45.0, -39.0,...
2  {'dst_nT': [-37.0, -41.0, -45.0, -39.0, -38.0,...
3  {'dst_nT': [-41.0, -45.0, -39.0, -38.0, -35.0,...
4  {'dst_nT': [-45.0, -39.0, -38.0, -35.0, -38.0,...
5  {'dst_nT': [-39.0, -38.0, -35.0, -38.0, -30.0,...


,dst_nT,time_enc
0,-45.0,"[0.0, 1.0, 0.017201575418260506, 0.99985204195..."
1,-37.0,"[0.25881904510252074, 0.9659258262890683, 0.01..."
2,-37.0,"[0.49999999999999994, 0.8660254037844387, 0.01..."
3,-41.0,"[0.7071067811865475, 0.7071067811865476, 0.017..."
4,-45.0,"[0.8660254037844386, 0.5000000000000001, 0.017..."
...,...,...
59,-10.0,"[0.258819045102521, -0.9659258262890682, 0.051..."
60,-9.0,"[1.2246467991473532e-16, -1.0, 0.0515843668694..."
61,-15.0,"[-0.2588190451025208, -0.9659258262890683, 0.0..."
62,-18.0,"[-0.4999999999999997, -0.8660254037844388, 0.0..."


In [9]:
df_truths = pd.DataFrame(training_sets["truths"])
df_truths

,0
0,-10.0
1,-9.0
2,-10.0
3,-13.0
4,-14.0
5,-16.0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_inputs, df_truths)